# Análise exploratória dos dados

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

In [2]:
load_dotenv()

diretorio_projeto = os.getenv("DIRETORIO_PROJETO") # Carregando o diretorio do projeto
 
data = pd.read_csv(diretorio_projeto + "/data/data.csv") # Carregando a base de dados original

df = data.copy() # Criando uma copia visando não modificar os dados iniciais

# Modelos de Machine Learning

In [3]:
from sklearn.model_selection import train_test_split

# Criando as variáveis X e Y para treinamento do modelo
x = df.drop('mauReal', axis = 1)
y = df['mauReal']

# Dividindo os dados em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, random_state = 42)

In [4]:
# Função para calcular MedAPE
def median_absolute_percentage_error(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
# Lista de modelos a serem testados
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "SVR": SVR()
}

Aqui vamos calcular as métricas MAPE, medAPE e RMSE tanto nos dados de treino e teste visando observar o desempenho dos modelos. Essa comparação poderá nos ajudar a fazer uma busca inicial do melhor modelo e depois filtrá-los fazendo uma melhoria com os hiperparâmetros.

A decisão de testar os modelos acima se deve ao fato de que temos modelos mais simples como regressão linear mas também modelos mais complexos como florestas aleatórias. Ressalto aqui que poderíamos ter testado diversos outros algoritmos, como redes neurais, KNN, etc. Porém, acredita-se fortemente que para esse problema, um modelo como floresta aleatória tem poder suficiente para ser utilizado. Não sendo necessária a utilização de modelos mais complexos como redes neurais. Em outros tipos de problemas esses algoritmos deveriam ser utilizados.

In [6]:
# Lista para armazenar as métricas
metrics = []

# Treinar modelos e calcular métricas
for name, model in tqdm(models.items()):
    model.fit(x_train, y_train)
    
    # Previsões
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Cálculo de métricas
    train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
    train_medape = median_absolute_percentage_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
    test_medape = median_absolute_percentage_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # Adicionar ao DataFrame
    metrics.append({
        "Model": name,
        "Train MAPE": train_mape,
        "Train MedAPE": train_medape,
        "Train RMSE": train_rmse,
        "Test MAPE": test_mape,
        "Test MedAPE": test_medape,
        "Test RMSE": test_rmse,
    })

# Converter para DataFrame
metrics_df = pd.DataFrame(metrics)

100%|██████████| 4/4 [01:32<00:00, 23.08s/it]


In [7]:
# Avaliando as métricas
metrics_df

,Model,Train MAPE,Train MedAPE,Train RMSE,Test MAPE,Test MedAPE,Test RMSE
0,Linear Regression,3.789551,81.640041,2.092886e+06,4.008198,81.610170,2.010108e+06
1,Random Forest,0.195930,0.104133,1.845751e+05,0.347273,0.299511,3.597777e+05
2,Decision Tree,0.000000,0.000000,0.000000e+00,0.409003,0.283083,6.051557e+05
3,SVR,3.438411,88.509615,5.195937e+06,3.695814,88.909668,5.151302e+06


Observa-se que os modelos de regressão como regressão linear e SRV tiveram um desempenho parecido. Porém, o respectivo desempenho é pior quando comparado aos modelos de árvore e de ensemble.

Porém, o modelo de árvore de decisão apresentou em sua maioria, um erro 0% para várias métricas, o que **sugere um possível overfitting**.

Sendo assim, seguiremos com uma abordagem de melhoria no modelo de **floresta aleatória**.

## Melhoria no modelo de Random Forest

Vamos definir os dois parâmetros principais para fazer a varredura e melhoria do modelo. Faremos a varredura do número de árvores e a profundidade máxima de cada árvore. Primeiro motivo desta escolha é que são os dois hiperparâmetros principais, gerando os melhores resultados com pouco esforço; segundo motivo é que fazer uma varredura com outros hiperparâmetros geraria um custo computacional muito grande, o que nesse momento não seria de fato importante. Vale ressaltar que em um projeto oficial, poderíamos utilizar várias abordagens para fazer essa varredura, usando apenas esses hiperparâmetros ou outros mais. Tudo dependeria do projeto, do tempo, e das necessidades.

In [8]:
# Modelo base
rf = RandomForestRegressor(random_state = 42)

# Definir o grid de hiperparâmetros para a varredura
param_grid = {
    'n_estimators': [50, 100, 200],          # Número de árvores
    'max_depth': [None, 10, 20, 30],         # Profundidade máxima
}

# Configurar a busca em grid com validação cruzada
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 1, scoring = 'neg_mean_squared_error')

In [9]:
# Executar a busca
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [10]:
# Melhor combinação de hiperparâmetros
print("Melhores hiperparâmetros:", grid_search.best_params_)

Melhores hiperparâmetros: {'max_depth': 20, 'n_estimators': 50}


In [11]:
# Treinando o modelo final com os melhores parâmetros
best_rf = grid_search.best_estimator_

In [12]:
# Lista com o modelo a ser avaliado
models_best_rf = {
    "Random Forest": RandomForestRegressor(max_depth = 20, n_estimators = 50, random_state = 42)
}

# Lista para armazenar as métricas
metrics_best_rf = []

# Treinar modelos e calcular métricas
for name, model in tqdm(models_best_rf.items()):
    model.fit(x_train, y_train)
    
    # Previsões
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Cálculo de métricas
    train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
    train_medape = median_absolute_percentage_error(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    test_mape = mean_absolute_percentage_error(y_test, y_test_pred)
    test_medape = median_absolute_percentage_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # Adicionar ao DataFrame
    metrics_best_rf.append({
        "Model": name,
        "Train MAPE": train_mape,
        "Train MedAPE": train_medape,
        "Train RMSE": train_rmse,
        "Test MAPE": test_mape,
        "Test MedAPE": test_medape,
        "Test RMSE": test_rmse,
    })

# Converter para DataFrame
metrics_df_best_rf = pd.DataFrame(metrics_best_rf)

100%|██████████| 1/1 [00:07<00:00,  7.62s/it]


In [13]:
metrics_df_best_rf

,Model,Train MAPE,Train MedAPE,Train RMSE,Test MAPE,Test MedAPE,Test RMSE
0,Random Forest,0.191286,0.23976,192865.423843,0.348144,0.460846,372971.112339


# Exportando o modelo e suas configurações

In [14]:
import pickle

# Exportar o modelo treinado
with open(diretorio_projeto + '/models/melhor_modelo_rf.pkl', 'wb') as f:
    pickle.dump(best_rf, f)

In [15]:
import json

# Salvar o nome das colunas
colunas = {'X_columns': list(x.columns), 'y_column': 'target_column'}

# Salvar em um arquivo JSON
with open(diretorio_projeto + '/models/colunas.json', 'w') as f:
    json.dump(colunas, f)

In [16]:
# Criando o dicionário com os dados
dados = {
    'x_train': x_train,
    'y_train': y_train,
    'x_test': x_test,
    'y_test': y_test
}

# Salvando o dicionário em um arquivo pickle
with open(diretorio_projeto + '/models/dados_treinamento.pkl', 'wb') as f:
    pickle.dump(dados, f)